In [1]:
# [Cell 1: .env 로드 및 Agent 임포트]
# .env 파일 로드 (core.py가 로드하지만, 노트북 환경을 위해 한 번 더)
import sys
import os
from dotenv import load_dotenv

# 1. 현재 프로젝트 루트 디렉토리(jesafe/)의 절대 경로를 찾습니다.
# os.getcwd() (현재 폴더)의 부모 폴더('..')가 프로젝트 루트라고 지정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# 2. 이 경로를 파이썬 모듈 검색 경로(sys.path)에 추가합니다.
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"프로젝트 루트 경로 추가: {project_root}")

load_dotenv()
print(".env 로드 완료:", os.getenv("DATABASE_URL") is not None)

프로젝트 루트 경로 추가: /Users/lwj0831/jesafe
.env 로드 완료: True


In [2]:
# [Cell 2: 메인 Agent 동기 테스트 (RunnableWithMessageHistory 사용)]
from src.services import main_agent_executor
from src.data_loader import _load_jeju_coords

# 테스트 전에 CSV 데이터를 메모리에 로드합니다.
_load_jeju_coords()

# 테스트에 사용할 고유 세션 ID
session_id = "test_session_notebook_123"

# --- 1. 날씨 질문 테스트 ---
# (프롬프트 규칙에 따라 '제주도'가 '제주시'로 변환되어야 함)
query1 = "오늘 제주도 날씨 어때?"
print(f"User: {query1}")

response1 = main_agent_executor.invoke(
    {"input": query1},
    config={"configurable": {"session_id": session_id}}
)
print(f"AI: {response1['output']}\n")
# (agent_with_history가 이 대화를 session_id에 자동 저장)


# --- 2. 사고 통계 질문 테스트 ---
query2 = "제주시에서 낙상 사고가 몇 건이야?"
print(f"User: {query2}")

response2 = main_agent_executor.invoke(
    {"input": query2},
    config={"configurable": {"session_id": session_id}}
)
print(f"AI: {response2['output']}\n")
# (agent_with_history가 이 대화를 session_id에 자동 저장)


# --- 3. (중요) 대화 맥락(Context) 질문 테스트 ---
# (이전 질문의 '제주시'를 기억하고 '제주시' 날씨를 답해야 함)
query3 = "그럼 그 지역 날씨는 어때?"
print(f"User: {query3}")

response3 = main_agent_executor.invoke(
    {"input": query3},
    config={"configurable": {"session_id": session_id}}
)
print(f"AI: {response3['output']}\n")

Initializing Compression Retriever (Ensemble + Filters)...
Initializing PGVector connection to collection: tourism_docs...
Initializing CacheBackedEmbeddings...
Initializing BM25Retriever (from PGVector's stored documents)...
벡터스토어에서 문서 로드 중...
✅ 벡터스토어에서 146개 문서 로드 완료


Quantization is not supported for ArchType::neon. Fall back to non-quantized model.
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


✅ BM25 Tokenizer 초기화 완료. 불용어 675개 적용됨.
BM25Retriever initialization complete.
Compression Retriever initialization complete.
✅ [data_loader] 제주도 (KMA 격자) DB 로드 완료 (경로: /Users/lwj0831/jesafe/data/kma_jeju_grid_info.csv)
User: 오늘 제주도 날씨 어때?


KeyError: "Input to ChatPromptTemplate is missing variables {'chat_history'}.  Expected: ['agent_scratchpad', 'chat_history', 'input'] Received: ['input', 'intermediate_steps', 'agent_scratchpad']\nNote: if you intended {chat_history} to be part of the string and not a variable, please escape it with double curly braces like: '{{chat_history}}'."

In [ ]:
# [Cell 3: RAG 임베딩 및 리트리버 전체 테스트]
import sys
import os

# (Cell 1과 동일) 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.core import get_vector_store, get_bm25_retriever
from src.core import _compression_retriever_instance

print("Connecting to Vector Store & Initializing Retrievers...")
try:
    # 앱 시작 시점과 동일하게 리소스 초기화
    vector_store = get_vector_store()
    bm25_retriever = get_bm25_retriever()
    
    # 테스트 쿼리
    query = "제주도 오름 추천해줘"
    print(f"\nTest Query: '{query}'")
    
    # -----------------------------------------------
    # 테스트 1: PGVector (의미기반 검색)
    # -----------------------------------------------
    print("\n--- 1. PGVector (Similarity Search) Test ---")
    vector_results = vector_store.similarity_search(query, k=3)
    print(f"Found {len(vector_results)} Chunks:")
    for i, doc in enumerate(vector_results):
        print(f"  [Vec {i+1}] {doc.page_content[:100]}...")

    # -----------------------------------------------
    # 테스트 2: BM25 (키워드 기반 검색)
    # -----------------------------------------------
    print("\n--- 2. BM25 (Keyword Search) Test ---")
    bm25_results = bm25_retriever.invoke(query) 
    print(f"Found {len(bm25_results)} Chunks:")
    for i, doc in enumerate(bm25_results):
        print(f"  [BM25 {i+1}] {doc.page_content[:100]}...")

    # -----------------------------------------------
    # 테스트 3: Ensemble (최종 앙상블)
    # -----------------------------------------------
    print("\n--- 3. Ensemble Retriever Test ---")
    ensemble_results = _compression_retriever_instance.base_retriever.invoke(query)
    print(f"Found {len(ensemble_results)} Chunks (after re-ranking):")
    for i, doc in enumerate(ensemble_results):
        print(f"  [Ensemble {i+1}] (Source: {doc.metadata.get('source', 'N/A')})")
        print(f"    {doc.page_content[:200]}...")

except Exception as e:
    print(f"\n❌ 테스트 실패: {e}")

Connecting to Vector Store & Initializing Retrievers...

Test Query: '제주도 오름 추천해줘'

--- 1. PGVector (Similarity Search) Test ---
Found 3 Chunks:
  [Vec 1] 별도봉은 제주시의 전경과 제주항을 오고가는 배들의 모습으로 볼 수 있는 곳으로 연인끼리 혹은 가족끼리 편하게 산책할 수 있는 아름다운 장소다.

제주시 화북1동 4472 탐방시간 ...
  [Vec 2] 제주시 구좌읍 종달리 산70 탐방시간 : 2시간 주변여행 : 제주자연생태공원, 비자림, 청초밭

복합형

큰노꼬메오름

난이도상 #한라산 조망 #넓은 주차장 #2개의 분화구

오름...
  [Vec 3] 긴 소매 옷, 긴 바지는 필수!

승마 너른 벌판을 달려라

넓은 초원을 달리고 싶다면 제주에서의 승마를

추천한다. 말위에서 느끼는 제주의 바람은 또 다른 맛을

선사해 준다.
...

--- 2. BM25 (Keyword Search) Test ---
Found 3 Chunks:
  [BM25 1] Ver 1.1

제주도 공식 관광정보

VISIT JEJU

4

CONTENTS

오름이란?……………………………………6

오름 트레킹 가이드… ……………………7

초보자를 위한 ...
  [BM25 2] 오름 트레킹 문의 및 정보

제주관광정보센터 064-740-6000 www.visitjeju.net

제주특별자치도 오름 탐방 064-120 https://gis.jeju.go.k...
  [BM25 3] 별도봉은 제주시의 전경과 제주항을 오고가는 배들의 모습으로 볼 수 있는 곳으로 연인끼리 혹은 가족끼리 편하게 산책할 수 있는 아름다운 장소다.

제주시 화북1동 4472 탐방시간 ...

--- 3. Ensemble Retriever Test ---
Found 5 Chunks (after re-ranking):
  [Ensemble 1] (Source: data/tourism_docs/

In [8]:
# [Cell 4: EmbeddingsFilter 상세 디버깅]
import sys
import os
import numpy as np
from langchain_community.utils.math import cosine_similarity

# 1. (Cell 1과 동일) 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 2. src/core.py에서 핵심 컴포넌트 임포트
from src.core import get_compression_retriever, get_cached_embedder

# 3. 테스트 쿼리 정의
query = "제주에 김종국이 갔다온 맛집 알려줘"
similarity_threshold = 0.80 # ⬅️ core.py에 설정된 임계값

print(f"Query: {query}")
print(f"Similarity Threshold: {similarity_threshold}")
print("-" * 30)

try:
    # 4. 리트리버 및 임베더 로드
    compression_retriever = get_compression_retriever()
    # 4-1. 압축 전 '기본' 리트리버 (EnsembleRetriever)
    base_retriever = compression_retriever.base_retriever
    # 4-2. 점수 계산에 사용할 임베더
    embedder = get_cached_embedder()

    # 5. (압축 전) 앙상블 리트리버가 찾은 문서 원본 가져오기
    print("Fetching docs from EnsembleRetriever (before filtering)...")
    retrieved_docs = base_retriever.invoke(query)
    
    if not retrieved_docs:
        print("EnsembleRetriever가 문서를 찾지 못했습니다.")
    else:
        print(f"Found {len(retrieved_docs)} docs. Calculating similarity scores...")
        
        # 6. (핵심) 쿼리와 각 문서의 임베딩 점수 수동 계산
        query_embedding = embedder.embed_query(query)
        doc_embeddings = embedder.embed_documents([doc.page_content for doc in retrieved_docs])
        
        # 7. 코사인 유사도 계산
        scores = cosine_similarity([query_embedding], doc_embeddings)[0]
        
        # 8. 결과 출력
        for i, (doc, score) in enumerate(zip(retrieved_docs, scores)):
            is_passed = score >= similarity_threshold
            
            print(f"\n[Doc {i+1}] Score: {score:.4f} (Threshold: {similarity_threshold})")
            print(f"  -> PASSED: {is_passed} ⬅️")
            print(f"  -> Content: {doc.page_content[:150]}...")

except Exception as e:
    print(f"\n❌ 테스트 실패: {e}")

Query: 제주에 김종국이 갔다온 맛집 알려줘
Similarity Threshold: 0.8
------------------------------
Fetching docs from EnsembleRetriever (before filtering)...
Found 9 docs. Calculating similarity scores...

[Doc 1] Score: 0.8507 (Threshold: 0.8)
  -> PASSED: True ⬅️
  -> Content: 대정의 들을 병풍처럼 에워싸고 있어 정상에서 한라산부터 산방산까지 서귀포 일대를 전망할 수 있으며, 걸어서 30분 정도로 쉽게 오를 수 있는 오름







K-CONTENTS | K-FOOD

# K-FOOD

**K-POP 가수와 각종 드라마, 방송에서 배우들이 ...

[Doc 2] Score: 0.8234 (Threshold: 0.8)
  -> PASSED: True ⬅️
  -> Content: ## 가시아방국수 GASIA BANG NOODLE
- **주소**: 제주 서귀포시 성산읍 섭지코지로 10  
- **가격**: 고기국수, 비빔국수 9,000원, 커플세트 36,000원  
두툼하고 부드러운 돔베고기가 올라간 고기국수가 인기 메뉴이며 수요미식회, 6시 내...

[Doc 3] Score: 0.8500 (Threshold: 0.8)
  -> PASSED: True ⬅️
  -> Content: # 오메기떡
차조 반죽으로 빚는 제주의 전통 떡이다. 통팥에 버무려진 떡 속에 달콤한 팥 앙금이 들어있는 것이 기본. 각종 매스컴에 알려진 유명 떡집은 문전성시를 이룬다. 요즘은 견과류나 과일 등을 응용한 다양한 맛의 오메기떡도 여행객을 유혹한다.

# 보말칼국수
고둥...

[Doc 4] Score: 0.7932 (Threshold: 0.8)
  -> PASSED: False ⬅️
  -> Content: ## 제주절물자연휴양림  
휴양과 치유의 숲  
* 위치: 제주시 명림로 584  


In [ ]:
# [Cell 5: DDGS 웹 검색 결과 수동 테스트]
import sys
import os

# 1. (Cell 1과 동일) 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 2. ddgs 라이브러리 직접 임포트
try:
    from ddgs import DDGS
except ImportError:
    print("❌ 'ddgs' 패키지가 설치되지 않았습니다. 'poetry add ddgs'를 실행하세요.")

# 3. 테스트 쿼리
query = "제주도 7월 축제"

print(f"Testing DDGS() with query: '{query}'")
print(f"Requesting max_results=3...")
print("-" * 30)

try:
    # 4. src/tools.py의 핵심 로직 직접 실행
    # (DDGS()는 컨텍스트 매니저(with) 사용을 권장)
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
    
    # 5. 결과 확인
    if not results:
        print("❌ 검색 결과가 없습니다.")
    else:
        print(f"✅ {len(results)}개의 결과를 성공적으로 가져왔습니다:")
        
        for i, res in enumerate(results):
            print(f"\n[Result {i+1}]")
            print(f"  Title: {res.get('title')}")
            print(f"  Snippet: {res.get('body', '')[:150]}...")
            print(f"  URL: {res.get('href')}")

except Exception as e:
    print(f"❌ 테스트 실패: {e}")

Testing DDGS() with query: '제주도 7월 축제'
Requesting max_results=3...
------------------------------
✅ 3개의 결과를 성공적으로 가져왔습니다:

[Result 1]
  Title: 제주시, 제주도 , 대한민국 월별 날씨 | AccuWeather
  Snippet: Get the monthly weather forecast for 제주시, 제주도 , 대한민국, including daily high/low, historical averages, to help you plan ahead....
  URL: https://www.accuweather.com/ko/kr/jeju/224209/july-weather/224209

[Result 2]
  Title: 제주도 11 월 축제 한눈에 보기! | Brunch Story
  Snippet: 태진아와 워너원, AOA, 레드벨벳, EXID, 오마이걸, 모모랜드, 나인뮤지스, 여자친구 등 총 28팀이 참가하는 제주도 최대 K팝 행사로 기대를 모으고 있답니다. 제주종합경기장 주경기장. 제주특별자치도 제주시 서광로2길 24....
  URL: https://brunch.co.kr/@gorrajeju/298

[Result 3]
  Title: 중3 수학여행 장기자랑 제주도 서연 | TikTok
  Snippet: #수학여행 #08 #07 # 제주도 #ootd #fyp 제주도 수학여행 코디 아이디어와 팁. 여름 제주도 수학여행을 위한 스타일과 코디를 소개합니다....
  URL: https://www.tiktok.com/discover/중3-수학여행-장기자랑-제주도-서연


In [7]:
#!/usr/bin/env python3
"""
리트리버 성능 디버깅 스크립트
각 단계별 결과를 확인하여 관련성 높은 청크를 놓치는 문제를 파악합니다.
"""

import sys
from src.core import debug_retriever_pipeline

# 테스트 쿼리들
test_queries = [
    "제주 고등어의 조림과 구이가 특별한 이유는 무엇일까요?",
    "제주 고등어 조림과 구이가 특별한 이유"
]

if __name__ == "__main__":
    print("\n🚀 리트리버 디버깅 테스트 시작\n")
    
    for query in test_queries:
        docs = debug_retriever_pipeline(query)
        
        # 원하는 답변이 포함되어 있는지 확인
        print("\n✅ 검증:")
        expected_keywords = ["고등어", "조림", "구이"]
        found = False
        
        for doc in docs:
            for keyword in expected_keywords:
                if keyword in doc.page_content:
                    print(f"   ✓ 예상 키워드 '{keyword}' 발견!")
                    found = True
                    break
        
        if not found:
            print(f"   ✗ 예상된 청크를 찾지 못함")
        
        print("\n" + "-"*60 + "\n")


🚀 리트리버 디버깅 테스트 시작


🔍 디버깅: 쿼리 = '제주 고등어의 조림과 구이가 특별한 이유는 무엇일까요?'

[Stage 1] BM25 검색 결과: 5개
  1. 대정의 들을 병풍처럼 에워싸고 있어 정상에서 한라산부터 산방산까지 서귀포 일대를 전망할 수 있으며, 걸어서 30분 정도로 쉽게 오를 수 있는 오...
  2. ## 김밥 김밥은 특별한 음식이 아니라고 할 분도 있겠다.   하지만 제주에서만 맛볼 수 있는 특별한 김밥들도 있다.   수많은 연예인들까지 인...
  3. 대한민국 최장 코스인 1,947m의 카트 트랙 코스를 자랑하며, 제주 남쪽 한라산의 풍경을 담은 아름다운 레이싱 코스의 익스트림 서킷과 온 가족...
  4. # 오메기떡 차조 반죽으로 빚는 제주의 전통 떡이다. 통팥에 버무려진 떡 속에 달콤한 팥 앙금이 들어있는 것이 기본. 각종 매스컴에 알려진 유명...
  5. # 수제햄버거 간편해 보이지만 결코 가벼운 맛은 아닌 수제햄버거 맛집도 많다. 제주 로컬 재료들을 이용하거나 자기만의 특별한 레시피로 승부하는 ...

[Stage 2] Vector 검색 결과: 5개
  1. ----  # 말고기구이  지방이 적은 말고기는 센 불에 살짝 익혀 먹는 것이 맛을 살리는 포인트다. 노릇하게 구어 한 점 먹는 순간 고소한 맛...
  2. Culture Jeju  # 생활 속 제주어  제주는 타 지역보다 한국어의 고형(古形)을 많이 유지하고 있는 동시에   제주도만의 고유한 어휘나...
  3. # 깅이죽 ‘깅이’는 ‘게’의 제주어. 5월의 별미인 깅이죽은 깅이를 돌절구에 찧으면 나오는 진한 즙에 물을 넣고 채에 걸러낸 국물에 서서히 끓...
  4. > “가을에 기름이 올라 겨울까지 맛있다”       꼭 맛봐야 할 고등어 요리  # 고등어회 수족관에 살아 있는 고등어로 회를 떠서 비린맛이 ...
  5. 깅이를 물과 함께 통째로 곱게 갈아 국물을 받은 후 쌀을 넣어 끓인 죽으로 정성이 많이 가는 귀한 음식이다. 특유의 맛과 향을 지니고

In [1]:
# [Cell 6: Cohere Reranker 통합 및 성능 검증 테스트] (수정됨)
import sys
import os

# 1. 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.core import get_compression_retriever

print("🚀 Cohere Reranker 통합 테스트 시작...")

try:
    # 2. 리트리버 로드
    compression_retriever = get_compression_retriever()
    
    # 3. 테스트 쿼리
    query = "제주도 고기국수 맛집 추천해줘"
    print(f"\n🔍 Query: '{query}'")
    
    # 4. 검색 실행
    final_docs = compression_retriever.invoke(query)
    
    if not final_docs:
        print("\n❌ 검색 결과가 없습니다. (DB 연결이나 임계값을 확인하세요)")
    else:
        print(f"\n✅ 최종 반환된 문서: {len(final_docs)}개")
        print("=" * 60)
        
        has_score = False
        
        for i, doc in enumerate(final_docs):
            score = doc.metadata.get('relevance_score', None)
            source = doc.metadata.get('source', 'Unknown')
            
            score_str = f"{score:.4f}" if score is not None else "N/A"
            if score is not None: has_score = True
            
            content_preview = doc.page_content[:120].replace('\n', ' ')
            
            print(f"[Rank {i+1}] Relevance Score: {score_str}")
            print(f"  Source: {source}")
            print(f"  Content Preview: {content_preview}...")
            print("-" * 60)
            
        # 5. 결과 검증 메시지
        print("\n[검증 결과]")
        if has_score:
            print("🎉 성공: 문서 메타데이터에서 'relevance_score'가 확인되었습니다.")
            print("   Cohere Reranker가 정상적으로 작동 중입니다.")
        else:
            print("⚠️ 주의: 'relevance_score'가 발견되지 않았습니다.")
            print("   CohereRerank가 파이프라인에 제대로 추가되지 않았거나, API 호출이 누락되었을 수 있습니다.")

except Exception as e:
    print(f"\n❌ 에러 발생: {e}")
    if "API" in str(e) or "key" in str(e).lower():
        print("💡 힌트: .env 파일에 COHERE_API_KEY가 올바르게 설정되었는지 확인하세요.")

🚀 Cohere Reranker 통합 테스트 시작...
Initializing Compression Retriever (Ensemble + Filters)...
Initializing PGVector connection to collection: tourism_docs...
Initializing CacheBackedEmbeddings...
Initializing BM25Retriever (from PGVector's stored documents)...
벡터스토어에서 문서 로드 중...
✅ 벡터스토어에서 146개 문서 로드 완료


/Users/lwj0831/jesafe/.venv/lib/python3.11/site-packages/langchain_classic/embeddings/cache.py:58: UserWarning: Using default key encoder: SHA-1 is *not* collision-resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA-256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
  _warn_about_sha1_encoder()
Quantization is not supported for ArchType::

✅ BM25 Tokenizer 초기화 완료. 불용어 675개 적용됨.
BM25Retriever initialization complete.
Compression Retriever initialization complete.

🔍 Query: '제주도 고기국수 맛집 추천해줘'


neon. Fall back to non-quantized model.



✅ 최종 반환된 문서: 3개
[Rank 1] Relevance Score: 0.9961
  Source: /Users/lwj0831/jesafe/data/tourism_docs/제주한류컨텐츠.pdf
  Content Preview: ## 가시아방국수 GASIA BANG NOODLE - **주소**: 제주 서귀포시 성산읍 섭지코지로 10   - **가격**: 고기국수, 비빔국수 9,000원, 커플세트 36,000원   두툼하고 부드러운 돔베고기가...
------------------------------------------------------------
[Rank 2] Relevance Score: 0.9871
  Source: /Users/lwj0831/jesafe/data/tourism_docs/제주한류컨텐츠.pdf
  Content Preview: K-CONTENTS | K-Food  # K-CONTENTS | K-Food  ## 우다 UDA - **주소**: 제주 서귀포시 속골로 13-7   - **가격**: 음료 및 식사 9,000원 ~ 20,000원   ...
------------------------------------------------------------
[Rank 3] Relevance Score: 0.9885
  Source: /Users/lwj0831/jesafe/data/tourism_docs/제주공식관광가이드북.pdf
  Content Preview: # 갈치요리 제주도에서 갈치를 먹는 방법은 다양하다. 갈치 본연의 맛을 느낄 수 있는 갈치구이. 갈치회는 비리지 않고 씹는 맛이 고소해 인기가 좋다. 싱싱한 갈치살이 입안에서 살살 녹는 조림 또한 갈치요리의 별미. ...
------------------------------------------------------------

[검증 결과]
🎉 성공: 문서 메타데이터에서 'relevance_score'가 확인되었습니다.
   Cohere Reranker가 정상적으로 작동 